In [10]:
import joblib
import pandas as pd
from mecab import MeCab

In [12]:
mecab = MeCab()

# 불용어 목록 정의
stop_words_list = [
    # '동대구역', '위', '입구', '동대', '구역', '인근', '주자', '공간', '앞', '뒤', '안', '옆', '근처', '주변', '가까운', '지역', '골목',
    # '친절', '덕분', '기분', '좋', '인상', '만족', '강추', '최애', '영접', '소문', '사실', '참한', '맛있', '최고', '추천',
    # '검색', '때', '제공', '기본', '찬', '경우', '있', '겉', '속', '전반', '성비', '곳', '가능', '같', '손', '에너지',
    # '인사', '모습', '여행', '차', '없', '손님', '많', '사장', '지친', '더보기', '방문', '다음', '말', '시간', '후',
    # '분위기', '사람', '처음', '생각', '리뷰', '완전', '편하다', '점', '느낌',
    # '맛있다', '맛집', '맛', '굿', '짱', '괜찮다',
    # '주문', '집', '매장', '식사', '가격', '서비스', '포장', '주방', '식당', '메뉴', '음식',
    # '담', '간', '층', '전', '날', '동', '월', '요', '수', '표', '무', '것', '끝', '직', '등', '함', '또',
    # '다', '별거', '아쉽', '종가', '높', '보통', '가위', '그만큼', '바보', '신세계백화점', '대구', '리회', '레기' , '기존'
]
stop_words = set(stop_words_list)

def stem_processing(text):
    if not isinstance(text, str) or not text.strip(): return ""
    try:
        pos_data = mecab.pos(text)
        result = []
        target_tags = {'NNG', 'NNP'}
        for word, tag in pos_data:
            if word in stop_words: continue
            if tag in target_tags:
                result.append(word)
            elif tag == 'VA':
                processed_word = word + "다"
                if processed_word not in stop_words:
                    result.append(processed_word)
        return " ".join(result)
    except Exception as e:
        return ""

In [ ]:
# 로드할 파일명
model_filename_to_load = 'tuned_voting_model.joblib'
preprocessor_filename_to_load = 'preprocessor.joblib'

try:
    # 모델 로드
    loaded_model = joblib.load(model_filename_to_load)

    # 전처리기 로드
    loaded_preprocessor = joblib.load(preprocessor_filename_to_load)

    # --- 로드된 모델 및 전처리기 사용 ---

    # 1. 예측할 새로운 단일 데이터 준비 (딕셔너리 형태)
    #    실제 상황에 맞는 값으로 수정
    single_example_review = "친구랑 저녁에 방문했는데 고기 질도 좋고 직원분이 친절하게 다 구워주셔서 편하게 먹었어요. 주차도 편해요!"

    single_example_dict = {
        # 텍스트 특징 (stem_processing 함수 필요)
        'PROCESSED_REVIEW': stem_processing(single_example_review),

        # 수치형 특징 (예시 값)
        'WAIT_encoded': 0.0, # 대기 없음

        # 범주형 특징 (예시 값)
        'HOW_예약 없이 이용': True, 'HOW_예약 후 이용': False, 'HOW_정보 없음': False, 'HOW_포장·배달 이용': False,
        'TIME_밤에 방문': False, 'TIME_아침에 방문': False, 'TIME_저녁에 방문': True, 'TIME_점심에 방문': False,
        'PURPOSE_데이트': False, 'PURPOSE_비즈니스': False, 'PURPOSE_가족모임': False, 'PURPOSE_여행': False, 'PURPOSE_나들이': False, 'PURPOSE_기념일': False, 'PURPOSE_친목': True, 'PURPOSE_회식': False, 'PURPOSE_일상': False,
        'WITH_WHOM_아이': False, 'WITH_WHOM_친구': True, 'WITH_WHOM_친척・형제': False, 'WITH_WHOM_연인・배우자': False, 'WITH_WHOM_혼자': False, 'WITH_WHOM_기타': False, 'WITH_WHOM_부모님': False, 'WITH_WHOM_지인・동료': False,

        # 키워드 특징 (리뷰 내용 기반 또는 기본값 0)
        '음식이 맛있어요': 1, # '맛있어요' 언급
        '친절해요': 1, # '친절하게' 언급
        '재료가 신선해요': 0,
        '양이 많아요': 0,
        '가성비가 좋아요': 0,
        '매장이 청결해요': 0,
        '매장이 넓어요': 0,
        '특별한 메뉴가 있어요': 0,
        '고기 질이 좋아요': 1, # '고기 질도 좋고' 언급
        '인테리어가 멋져요': 0
    }

    # DataFrame 생성 (단일 행)
    new_data_to_predict = pd.DataFrame([single_example_dict])

    print(f"\n예측할 원본 리뷰: {single_example_review}")
    # print(f"예측용 입력 데이터 (DataFrame):\n{new_data_to_predict}") # 필요시 주석 해제

    # 2. 로드된 전처리기를 사용하여 데이터 변환
    new_data_processed = loaded_preprocessor.transform(new_data_to_predict)
    print(f"\n전처리된 데이터 형태: {new_data_processed.shape}")

    # 3. 로드된 모델을 사용하여 예측 수행
    new_prediction = loaded_model.predict(new_data_processed)[0] # 단일 결과 추출
    new_probabilities = loaded_model.predict_proba(new_data_processed)[0] # 단일 결과 확률 추출

    print("\n--- 예측 결과 ---")
    print(f"예측된 카테고리: {new_prediction}")

    # 상위 예측 확률 확인
    print("\n예측 확률 (상위 3개):")
    # model.classes_ 대신 loaded_model.classes_ 사용
    proba_list = sorted(zip(loaded_model.classes_, new_probabilities), key=lambda item: item[1], reverse=True)
    for i, (category, prob) in enumerate(proba_list[:3]):
        print(f"  {i+1}. {category}: {prob:.4f}")


except FileNotFoundError as e:
    print(f"오류: 저장된 파일을 찾을 수 없습니다. '{e.filename}' 경로를 확인하세요.")
except NameError as e:
    print(f"오류: 필요한 변수(feature_columns, stem_processing 등)가 정의되지 않았습니다. 이전 셀 실행을 확인하세요. 오류: {e}")
except Exception as e:
    print(f"파일 로드 또는 예측 중 오류 발생: {e}")

print("\n--- 로드 및 단일 예측 예시 완료 ---")


예측할 원본 리뷰: 친구랑 저녁에 방문했는데 고기 질도 좋고 직원분이 친절하게 다 구워주셔서 편하게 먹었어요. 주차도 편해요!

전처리된 데이터 형태: (1, 2379)

--- 예측 결과 ---
예측된 카테고리: 고기

예측 확률 (상위 3개):
  1. 고기: 0.8252
  2. 아시아/해산물: 0.0581
  3. 한식: 0.0437

--- 로드 및 단일 예측 예시 완료 ---


c:\Users\qwg52\.conda\envs\env312_cpu_v1\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\qwg52\.conda\envs\env312_cpu_v1\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
